In [ ]:
# https://www.youtube.com/watch?v=rwXVCELcrqU&list=PLgUwDviBIf0rf5CQf_HFt35_cF04d8dHN&index=4

In [20]:
from math import ceil, inf, log2
from typing import List


def formSegmentTree(arr: List[int]):
    n = len(arr)
    nLevels = ceil(log2(n))
    lengthOfSegmentTree = 2**(nLevels+1)
    seg = [0] * lengthOfSegmentTree
    lazy = [0] * lengthOfSegmentTree
    def build(ind: int, low: int, high: int):
        if low == high:
            seg[ind] = arr[low]
            return seg[ind]
        mid = (low + high) // 2
        build(2*ind + 1, low, mid)
        build(2*ind + 2, mid+1, high)
        seg[ind] = seg[2*ind+1] + seg[2*ind+2]

    build(0, 0, n - 1)

    def rangeUpdate(ind: int, low: int, high: int, l: int, r: int, add: int):
        if lazy[ind] != 0:
            seg[ind] += (high - low + 1) * lazy[ind]
            if low != high:
                lazy[2*ind+1] += lazy[ind]
                lazy[2*ind+2] += lazy[ind]
            lazy[ind] = 0
        
        # outside range
        if high < l or low > r:
            return 
        # inside range
        if low >= l and high <= r:
            seg[ind] += (high - low + 1) * add
            if low != high:
                lazy[2*ind+1] += add
                lazy[2*ind+2] += add
            return
        
        mid = (low + high) // 2
        rangeUpdate(2*ind+1, low, mid, l, r, add)
        rangeUpdate(2*ind + 2, mid+1, high, l, r, add)
        seg[ind] = seg[2*ind+1] + seg[2*ind+2]

    def queryLazy(ind: int, low: int, high: int, l:int, r:int):

        if lazy[ind] != 0:
            seg[ind] += (high - low + 1) * lazy[ind]
            if low != high:
                lazy[2*ind+1] += lazy[ind]
                lazy[2*ind+2] += lazy[ind]
            lazy[ind] = 0


        # inside range
        if low >= l and high <= r:
            return seg[ind]
        
        # outside range
        if high < l or low > r:
            return 0
        
        mid = (low + high) // 2
        left = queryLazy(2*ind+1, low, mid, l, r)
        right = queryLazy(2*ind+2, mid+1, high, l, r)
        return left + right
    
    def findSumOfRange(l:int, r:int):
        return queryLazy(0, 0, n-1, l, r)

    # updates value at index with new value v
    def updateAtIndex(i: int, v: int):
        # change value at index
        add = v - arr[i]
        arr[i] = v
        rangeUpdate(0, 0, n-1, i, i, add)

    # changes value at index by add
    def updateIndexBy(i: int, add: int):
        # change value at index
        arr[i] += add
        rangeUpdate(0, 0, n-1, i, i, add)

    def updateRangeInArr(l: int, r: int, add: int):
        for i in range(l, r + 1):
            arr[i] += add
        rangeUpdate(0, 0, n-1, l, r, add)
        
    return findSumOfRange, updateAtIndex, updateIndexBy, updateRangeInArr

In [21]:
#      0  1  2  3  4  5  6  7  8   9
arr = [8, 2, 5, 1, 4, 5, 3, 9, 6, 10]
findSumOfRange, updateAtIndex, updateIndexBy, updateRangeInArr = formSegmentTree(arr)

findSumOfRange(5, 7)

17

In [22]:
updateAtIndex(5, 6)
findSumOfRange(5, 7)

18

In [23]:
updateIndexBy(5, 1)
findSumOfRange(5, 7)

19

In [24]:
updateRangeInArr(5, 6, 1)
findSumOfRange(5, 7)

21

In [25]:
def formPrefixSum(arr: List[int]):
    n = len(arr)
    prefix_sum = [0] * (n+1)
    for i in range(1, n+1):
        prefix_sum[i] = prefix_sum[i-1] + arr[i-1]
    def get_sum(i, j):
        return prefix_sum[j + 1] - prefix_sum[i]
    return get_sum

In [26]:
n = len(arr)
rangeSumFromPrefixSum = formPrefixSum(arr)
print(rangeSumFromPrefixSum(5, 7))
for i in range(n):
    for j in range(i,n):
        # print(f'{i=} {j=} {findSumOfRange(i,j)=} {rangeSumFromPrefixSum(i,j)=}')
        if findSumOfRange(i, j) != rangeSumFromPrefixSum(i, j):
            raise Exception('results are different')

21
